# Bird Classification
#### with Clustering Experiment and Two Upsampling Implementations

The dataset has six classes, and contains info about six ecological bird groups:
*Swimming Birds.
*Wading Birds. *Terrestrial Birds. *Raptors. *Scansorial Birds. *Singing Birds.

Each sample/bird is comprised of ten measurements, the length and diameter of five bones.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import warnings
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('../input/birds-bones-and-living-habits/bird.csv')
df.head()

In [ ]:
df.drop(columns='id', inplace=True)
df.info()

In [ ]:
#visualize where NaN values are located in the dataset
plt.figure(figsize=(6,5))
sns.heatmap(df.isna());

These are the rows that have two or more NaN values:

In [ ]:
df[(df.isna().cumsum(axis=1).iloc[:,-1])>=2]

Let's erase them

In [ ]:
df.dropna(axis=0, thresh=10, inplace=True)

The two remaining NaN values turn out to belong to the majority class ('SO'), which contains double and more instances than most of the other classes, so we may just remove it.

In [ ]:
#drop all NaNs
df.dropna(how='any', inplace=True)

In [ ]:
#how many samples for each class
df['type'].value_counts()

In [ ]:
sns.pairplot(df.iloc[:,:-1]);

We'll draw a correlation heatmap to identify features that correlate highly with each other. For each pair of such features, we will remove one of them from the dataset, otherwise it's as if we use the same column twice.

We'll be watching out for correlation coefficients > 0.93, a somewhat arbitrarily chosen threshold, but certainly high enough as not to throw out important information.

In [ ]:
#correlation heatmap
plt.figure(figsize=(10,7))
sns.heatmap(df.iloc[:,:-1].corr(), annot=True, fmt='.2f');

Since the redundant columns that we'll drop previously contained many of the dataset's NaN values, we will re-import the dataset and first drop these columns before removing NaN values. Now the dataset contains 414 entries instead of 413.

In [ ]:
df=pd.read_csv('../input/birds-bones-and-living-habits/bird.csv')  \
.drop(columns=['id','ulnal','ulnaw','femw','tibw']).dropna(how='any')

In [ ]:
sns.pairplot(df, vars= df.columns[:-1], hue='type');

In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(df.iloc[:,:-1].corr(), annot=True, fmt='.2f');

In [ ]:
#distributions per class
def boxes(features):
    plt.figure(figsize=(12,5));
    plt.subplot(1,2,1);
    sns.boxplot(x='type', y=features[0], data=df);
    plt.title(features[0]);
    plt.subplot(1,2,2);
    sns.boxplot(x='type', y=features[1], data=df);
    plt.title(features[1]);

boxes(df.columns[:2])
boxes(df.columns[2:4])
boxes(df.columns[4:6])

The boxplots reveal that the distributions of the classes often fall within similar ranges. It's hard to identify a feature with high significance. The models we'll later train  probably won't reach high accuracy, as the data don't contain enough information relevant to class discrimination. 

The dataset is about the size of bones of different bird categories, but it seems different categories share similar bone sizes, making our classification task harder.

Let's draw a PCA scatterplot to inspect visually the degree of separability of the data

In [ ]:
#PCA
sc=StandardScaler()
x_sc=sc.fit_transform(df.iloc[:,:-1].values)

pca=PCA(n_components=2)
x_pc=pca.fit_transform(x_sc)

plt.figure(figsize=(10,7));
sns.scatterplot(x_pc[:,0],x_pc[:,1],hue=df['type']);

In the PCA scatterplot, using two Principal Components, the data don't seem well-separated. Would a higher number of Principal Components alleviate this, or is the data inherently hard to separate? Let's draw a graph of the explained variance of the Principal Components:

In [ ]:
pca=PCA()
pca.fit(x_sc)

sns.lineplot(x=np.arange(pca.components_.shape[0])+1, 
             y=pca.explained_variance_ratio_.cumsum());
plt.ylim(0.0,1.0);
plt.title('Cumulative explained variance of PCA');
plt.xlabel('number of principal component');
plt.ylabel('cumulative sum of explained variance');

In [ ]:
sns.barplot(x=np.arange(pca.components_.shape[0])+1, y=pca.explained_variance_ratio_, color='grey')
plt.title('Explained variance of PCA');
plt.xlabel('Number of principal component');
plt.ylabel('Explained variance');

print('Number of Principal Components needed to explain total variance:',
      pca.components_.shape[0] )
print('Cumulative explained variance of first two components:',
      pca.explained_variance_ratio_[:2].cumsum()[1])
print('Individual explained variance of first three components:',
      pca.explained_variance_ratio_[:3])

From the above data and measuments we see that the first two Components account for 92,53% of the total variance, and that adding a third Component would increase explained variance only by an insignificant 4,86%. 

Therefore, the problem doesn't lie in the number of Principal Components we chose, but in the inherent data distributions. This was also evident from the boxplots and the kde plots of the pairgrid we previously inspected.


In [ ]:
#PCA on raw data
pc=PCA(n_components=2)
x_dem=pc.fit_transform(df.iloc[:,:-1].values)
plt.figure(figsize=(10,7));
sns.scatterplot(x_dem[:,0], x_dem[:,1], hue=df['type']);

Interestingly, the raw-data PCA scatterplot doesn't look much different than the standardized-data PCA scatterplot. This must be because the data are in similar scales, measurements of length and diameter of bones.

Now, before we start applying machine learning models we will perform a little experiment. Since the data seems hard to separate, and since we already know the class each instance belongs to, it will be interesting to try out unsupervised clustering and see how it works out.

## *Clustering side experiment

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples

In [ ]:
#use 'elbow' method to find optimal cluster number
inertia=[]
for i in range(1,11):
    km=KMeans(n_clusters=i, random_state=33)
    km.fit_predict(x_sc)
    inertia.append(km.inertia_)
    
#inertia plot
sns.lineplot(range(1,11),inertia);
plt.title('Inertia');

From the 'elbow' method it seems that the optimal cluster number is five, although we know there are six classes. We will try with both five and six clusters.

In [ ]:
#five clusters
km=KMeans(n_clusters=5, random_state=33)
clusters=km.fit_predict(x_sc)

#Silhouette Graph
#code from Sebastian Raschka's book 'Python Machine Learning'
labels=np.unique(clusters)
n_clusters=labels.shape[0]
sils=silhouette_samples(x_sc,clusters,metric='euclidean')
y_ax_lower, y_ax_upper=0, 0
yticks=[]
plt.figure(figsize=(6,5))
for i,c in enumerate(labels):
    cluster_sil=sils[clusters==c]
    cluster_sil.sort()
    y_ax_upper +=len(cluster_sil)
    color=cm.jet(float(i)/n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper),
            cluster_sil, height=1.0,
            edgecolor='none', color=color)
    yticks.append((y_ax_lower+y_ax_upper)/2.)
    y_ax_lower+=len(cluster_sil)
silhouette_avg=np.mean(sils)
plt.axvline(silhouette_avg,color='red', linestyle='--')
plt.yticks(yticks,labels+1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.show();

#visualize clusters on PCA data
pca=PCA(n_components=2)
dec=pca.fit_transform(x_sc)
plt.figure(figsize=(9,6));
sns.scatterplot(dec[:,0],dec[:,1], hue=clusters);

In [ ]:
km=KMeans(n_clusters=6, random_state=33)
clusters=km.fit_predict(x_sc)

#Silhouette Graph
labels=np.unique(clusters)
n_clusters=labels.shape[0]
sils=silhouette_samples(x_sc,clusters,metric='euclidean')
y_ax_lower, y_ax_upper=0, 0
yticks=[]
plt.figure(figsize=(6,5))
for i,c in enumerate(labels):
    cluster_sil=sils[clusters==c]
    cluster_sil.sort()
    y_ax_upper +=len(cluster_sil)
    color=cm.jet(float(i)/n_clusters)
    plt.barh(range(y_ax_lower, y_ax_upper),
            cluster_sil, height=1.0,
            edgecolor='none', color=color)
    yticks.append((y_ax_lower+y_ax_upper)/2.)
    y_ax_lower+=len(cluster_sil)
silhouette_avg=np.mean(sils)
plt.axvline(silhouette_avg,color='red', linestyle='--')
plt.yticks(yticks,labels+1)
plt.ylabel('Cluster')
plt.xlabel('Silhouette coefficient')
plt.show();

#visualize clusters on PCA data
pca=PCA(n_components=2)
dec=pca.fit_transform(x_sc)
plt.figure(figsize=(9,6));
sns.scatterplot(dec[:,0],dec[:,1], hue=clusters);

The clustering only seems to work on the 'SO' class, and perhaps a little bit on the 'SW'. We have data that are very hard to cluster, and will probably be hard to classify.

Let's see

# Machine Learning

### Preprocessing

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# create train and test sets, data and labels
x=x_sc.copy()

enc=LabelEncoder()
y=enc.fit_transform(df['type'])
print ('original classes:',enc.classes_)

x_tr, x_ts, y_tr, y_ts=train_test_split(x,y,random_state=42)

Before we start training models, we must consider that the dataset is imbalanced, as shown below

In [ ]:
#graph class proportions in the dataset
plt.figure(figsize=(12,5));
plt.subplot(1,2,1);
plt.title('class_count');
sns.countplot(x='type', data=df, color='grey');
plt.subplot(1,2,2);
df['type'].value_counts().plot(kind='pie',autopct='%1.1f%%');
plt.title('class_proportions');

This will pose problems to our machine learning. To demonstrate this, below is how a classifier that makes random predictions would perform, as well as a classifier that classifies all samples as within one of the two majority classes

In [ ]:
#dummy classifications
np.random.RandomState(seed=7)
print('random selection accuracy:' ,
 accuracy_score(y,np.random.choice([0,1,2,3,4,5],(x.shape[0]))))

dum=DummyClassifier(strategy='uniform', random_state=7)
dum.fit(x,y)
print ('random classifier accuracy:',
      accuracy_score (y, dum.predict(x)))

print("")
print('majority class selection accuracy:' ,
 accuracy_score(y,np.random.choice([2,3],(x.shape[0]))))

A random classifier will yield accuracy ~17%.

A classifier that randomly assigns every sample to one of the two majority classes would yield accuracy ~27%, which is about half of the percentage of these two classes' presence on the dataset. If the classifier would learn correctly to distinguish between these two classes, and also classified other samples to the class (from the two majority classes) they're closer to,  accuracy would double at ~54%.

With that in mind, lets see how two models, trained on the imbalanced dataset, perform.

In [ ]:
#training on the imbalanced data
lr=LogisticRegression()
lr.fit(x_tr,y_tr)
print('LogistReG accuracy:', accuracy_score(y_ts, lr.predict(x_ts) ))
print('LogistReG f1_score:', f1_score(y_ts, lr.predict(x_ts),average='macro' ))

rf=RandomForestClassifier(1000, max_depth=8)
rf.fit(x_tr,y_tr)
print('RandForest accuracy:', accuracy_score(y_ts,rf.predict(x_ts)))
print('RandForest f1-score:', f1_score(y_ts,rf.predict(x_ts), average='macro'))

Given the 56% accuracy we'd expect from a classifier that only classifies samples in the two majority classes, and classifies data belonging to these two classes correctly, the performance of the two models isn't mindblowing.

To deal with this, we will resample the training set.

We will keep the original dataset, but we will also create two balanced ones, and we'll try our models on all three datasets and compare the results. The first balanced dataset will be generated simply by oversampling the minority classes. For the second one, 80% of the additional data will be duplicated samples, and 20% will be generated by averaging samples from the minority classes. (Specifically, four samples will be duplicates of the original entries and the fifth will be the mean of the previous four, continuing this until each class expands and reaches the same number of samples as the majority class)

In [ ]:
#first upsampling implementation
#mere duplicating samples from minority classes

print('x_tr.shape=', x_tr.shape)
print('y_tr.shape=', y_tr.shape)
print('')


#initializations
majority_class=np.argmax(np.bincount(y_tr))
length_majority_class = np.max(np.bincount(y_tr))
additional_samples=np.array([])
additional_labels=np.array([])


for i in np.unique(y_tr):

    if i != majority_class:
        length_this_class=np.bincount(y_tr)[i]
        wanted=length_majority_class-length_this_class
        print('class:',i)
        print('%d additional samples' % wanted)
        indexes=np.argwhere(y_tr==i)
        indexes=np.reshape(indexes, (len(indexes),))
        choices=np.random.choice(indexes,size=wanted)
        temp_x=x_tr[choices]
        additional_samples=np.append(additional_samples,temp_x, axis=None)
        additional_labels=np.append(additional_labels,np.full(wanted,i), axis=None)
        

#new samples
print('')
additional_samples=np.reshape(additional_samples,(int(len(additional_samples)/6),6))
print('new_samples.shape=', additional_samples.shape)
print('new_labels.shape=', additional_labels.shape)

#new x_tr --> x_tr1
flat_x = np.append(x_tr,additional_samples)
x_columns_number = x_tr.shape[1]
new_row_number = int(len(flat_x)/x_columns_number)
x_tr1 = np.reshape(flat_x,(new_row_number,x_columns_number))

#new y_tr --> y_tr1
flat_y = np.append (y_tr, additional_labels)
new_shape = (len(y_tr)+len(additional_labels),)
y_tr1 = np.reshape(flat_y, new_shape)
y_tr1 = y_tr1.astype('int64')

print('')
print ('x_tr1.shape=',x_tr1.shape)
print ('y_tr1.shape=',y_tr1.shape)
print('')
print('number of samples for each class, before resampling:', np.bincount(y_tr))
print('number of samples for each class, after resampling: ', np.bincount(y_tr1))

In [ ]:
#second upsampling implementation
#duplicating samples plus generating synthetic ones
#for every five samples, the fifth will be the average of the previous four
#the previous four are duplicates of samples present in the original dataset

print('x_tr.shape=', x_tr.shape)
print('y_tr.shape=', y_tr.shape)
print('')


#initializations
majority_class=np.argmax(np.bincount(y_tr))
length_majority_class = np.max(np.bincount(y_tr))
additional_samples=[]
additional_labels=[]
counter=0


for i in np.unique(y_tr):

    if i != majority_class:
        length_this_class=np.bincount(y_tr)[i]
        wanted=length_majority_class-length_this_class
        n_new_samples=0
        indexes=np.argwhere(y_tr==i)
        indexes=np.reshape(indexes, (len(indexes),))
        print('class:',i)
        print('%d additional samples' % wanted)
        while n_new_samples < wanted:
            n_new_samples +=1
            choice_index=np.random.choice(indexes,size=1)
            choice_sample= x_tr [choice_index]
            additional_samples=np.append(additional_samples,choice_sample)
            additional_labels=np.append(additional_labels,i)
            counter +=1
            if counter == 4:
                additional_samples=np.reshape(additional_samples,
                                              (int(len(additional_samples)/6),6))
                new_sample=np.mean(additional_samples[-4:,:],axis=0)
                additional_samples=np.append(additional_samples,[new_sample])
                additional_labels=np.append(additional_labels,i)
                counter=0
                n_new_samples += 1
        if n_new_samples > wanted:
            additional_samples=np.reshape(additional_samples,(int(len(additional_samples)/6),6))
            redundant = n_new_samples - wanted
            additional_samples = additional_samples[:-redundant,:].copy()
            additional_labels=additional_labels[:-redundant]
      
        

#new samples
print('')
print('new_samples.shape=', additional_samples.shape)
print('new_labels.shape=', additional_labels.shape)

#new x_tr --> x_tr2
flat_x = np.append(x_tr,additional_samples)
x_columns_number = x_tr.shape[1]
new_row_number = int(len(flat_x)/x_columns_number)
x_tr2 = np.reshape(flat_x,(new_row_number,x_columns_number))

#new y_tr --> y_tr2
flat_y = np.append (y_tr, additional_labels)
new_shape = (len(y_tr)+len(additional_labels),)
y_tr2 = np.reshape(flat_y, new_shape)
y_tr2 = y_tr2.astype('int64')

print('')
print ('x_tr2.shape=',x_tr2.shape)
print ('y_tr2.shape=',y_tr2.shape)
print('')
print('number of samples for each class, before resampling:', np.bincount(y_tr))
print('number of samples for each class, after resampling: ', np.bincount(y_tr2))

In [ ]:
#training on the new, balanced data
lr=LogisticRegression()
lr.fit(x_tr1,y_tr1)
print('LogistReG accuracy:', accuracy_score(y_ts, lr.predict(x_ts) ))
print('LogistReg f1-score:', f1_score(y_ts,lr.predict(x_ts), average='macro'))

rf=RandomForestClassifier(1000, max_depth=8)
rf.fit(x_tr1,y_tr1)
print('RandForest accuracy:', accuracy_score(y_ts,rf.predict(x_ts)))
print('RandForest f1-score:', f1_score(y_ts,rf.predict(x_ts), average='macro'))

In [ ]:
#training on the new, balanced and synthetic data
lr=LogisticRegression()
lr.fit(x_tr2,y_tr2)
print('LogistReG accuracy:', accuracy_score(y_ts, lr.predict(x_ts) ))
print('LogistReg f1-score:', f1_score(y_ts,lr.predict(x_ts), average='macro'))

rf=RandomForestClassifier(1000, max_depth=8)
rf.fit(x_tr2,y_tr2)
print('RandForest accuracy:', accuracy_score(y_ts,rf.predict(x_ts)))
print('RandForest f1-score:', f1_score(y_ts,rf.predict(x_ts), average='macro'))

Now, after we generated the new data and did a quick test with Logistic Regression and Random Forest, let's go through the entire process of model training

### Training Models (on original, imbalanced training set)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier


#dataframes to store experimental results, for later comparison
results=pd.DataFrame([], 
    columns=['model', 'parameters','accuracy','precision','recall','F1-score'])
results1=pd.DataFrame([], 
    columns=['model', 'parameters','accuracy','precision','recall','F1-score'])
results2=pd.DataFrame([], 
    columns=['model', 'parameters','accuracy','precision','recall','F1-score'])
majority_vote_results=pd.DataFrame([], 
    columns=['training dataset','accuracy','precision','recall','F1-score' ])

In [ ]:
#Naive Bayes
gnb=GaussianNB()
gnb.fit(x_tr,y_tr)
y_pred=gnb.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Gaussian NB', 'default', accuracy,precision,recall,f1]],columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]


In [ ]:
#KNN
knn=KNeighborsClassifier()
knn.fit(x_tr,y_tr)
y_pred=knn.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['KNN', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#SVM
svm=SVC()
svm.fit(x_tr,y_tr)
y_pred=svm.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['SVM', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
# SGD
sgd=SGDClassifier()
sgd.fit(x_tr,y_tr)
y_pred=sgd.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['SGD', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#SGD with different parameters
sgd_m=SGDClassifier(alpha=0.01, loss='modified_huber', penalty='l1')
sgd_m.fit(x_tr,y_tr)
y_pred=sgd_m.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['SGD 2', 'alpha=0.01, loss=modified_huber, penalty=l1', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
# Logistic Regression
lr=LogisticRegression()
lr.fit(x_tr,y_tr)
y_pred=lr.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Logistic Regression', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#Decision Tree
tree=DecisionTreeClassifier(max_depth=5, random_state=42)
tree.fit(x_tr,y_tr)
y_pred=tree.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Decision Tree', 'max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#Random Forest
rf=RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=42)
rf.fit(x_tr,y_tr)
y_pred=rf.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Random Forest', 'n_estimators=1000, max_depth=7', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#Ada Boost
tree_ada=DecisionTreeClassifier(criterion='entropy',random_state=1, max_depth=5)
ada=AdaBoostClassifier(base_estimator=tree_ada,n_estimators=1000,learning_rate=0.1, random_state=5)
ada.fit(x_tr,y_tr)
y_pred=ada.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Ada Boost', 'criterion=entropy, max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
#Gradient Boosted Tree
gb=GradientBoostingClassifier(random_state=5, n_estimators=1000, max_depth=4)
gb.fit(x_tr,y_tr)
y_pred=gb.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results=results.append(pd.DataFrame([['Gradient Boosted Classifier', 'n_estimators=1000, max_depth=4',
                                      accuracy,precision,recall,f1]],
                                    columns=list(results.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results.iloc[-1:]

In [ ]:
results= results.reset_index().drop(columns='index')
results

Clearly the Gradient Boosted Tree is superior, followed by Random Forest and AdaBoost. Let's implement a majority vote (leaving Naive Bayes out) for comparison

In [ ]:
# majority vote implementation
pred_knn = knn.predict(x_ts) 
pred_svm = svm.predict(x_ts)
pred_sgd_m= sgd_m.predict(x_ts)
pred_lr= lr.predict(x_ts)
pred_tree= tree.predict(x_ts)
pred_rf= rf.predict(x_ts)
pred_ada=ada.predict(x_ts)
pred_gb=gb.predict(x_ts)

mode=[]
for i in np.arange(y_ts.shape[0]):
    votes=np.array([pred_knn[i], pred_svm[i], pred_sgd_m[i],pred_lr[i],
                    pred_tree[i], pred_rf[i], pred_ada[i], pred_gb[i]])
    bincount=np.bincount(votes)
    mode=np.append(mode,np.argmax(bincount)) #mode is the most common vote, the majority vote


majority_vote_results=majority_vote_results.append(pd.DataFrame([['original',
            accuracy_score(y_ts, mode),precision_score(y_ts, mode,average='macro'),
            recall_score(y_ts, mode,average='macro'),f1_score(y_ts, mode,average='macro')]],
            columns=list(majority_vote_results.columns)))
majority_vote_results.iloc[-1:]

> It surpasses the Gradient Boosted Tree.

Let's try our models on the two resampled training sets

### Training Models (on first resampled training set)

In [ ]:
#Naive Bayes
gnb1=GaussianNB()
gnb1.fit(x_tr1,y_tr1)
y_pred=gnb1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Gaussian NB', 'default', accuracy,precision,recall,f1]],columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]


In [ ]:
#KNN
knn1=KNeighborsClassifier()
knn1.fit(x_tr1,y_tr1)
y_pred=knn1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['KNN', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#SVM
svm1=SVC()
svm1.fit(x_tr1,y_tr1)
y_pred=svm1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['SVM', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#SGD
sgd1=SGDClassifier()
sgd1.fit(x_tr1,y_tr1)
y_pred=sgd1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['SGD', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#SGD with diferrent parameters
sgd_m1=SGDClassifier(alpha=0.01, loss='modified_huber', penalty='l1')
sgd_m1.fit(x_tr1,y_tr1)
y_pred=sgd_m1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['SGD 2', 'alpha=0.01, loss=modified_huber, penalty=l1', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#Logistic Regression
lr1=LogisticRegression()
lr1.fit(x_tr1,y_tr1)
y_pred=lr1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Logistic Regression', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#Decision Tree
tree1=DecisionTreeClassifier(max_depth=5, random_state=42)
tree1.fit(x_tr1,y_tr1)
y_pred=tree1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Decision Tree', 'max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#Random Forest
rf1=RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=42)
rf1.fit(x_tr1,y_tr1)
y_pred=rf1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Random Forest', 'n_estimators=1000, max_depth=7', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#Ada Boost
tree_ada1=DecisionTreeClassifier(criterion='entropy',random_state=1, max_depth=5)
ada1=AdaBoostClassifier(base_estimator=tree_ada1,n_estimators=1000,learning_rate=0.1, random_state=5)
ada1.fit(x_tr1,y_tr1)
y_pred=ada1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Ada Boost', 'criterion=entropy, max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
#Gradient Boosted Tree
gb1=GradientBoostingClassifier(random_state=5, n_estimators=1000, max_depth=4)
gb1.fit(x_tr1,y_tr1)
y_pred=gb1.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results1=results1.append(pd.DataFrame([['Gradient Boosted Classifier', 'n_estimators=1000, max_depth=4',
                                      accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results1.iloc[-1:]

In [ ]:
results1=results1.reset_index().drop(columns='index')
results1

Here too Gradient Boosted Tree excels on all metrics, followed by AdaBoost andRandom Forest. Let's implement a majority vote (leaving Naive Bayes out) for comparison

In [ ]:
# majority vote implementation
pred_knn1 = knn1.predict(x_ts) 
pred_svm1 = svm1.predict(x_ts)
pred_sgd_m1= sgd_m1.predict(x_ts)
pred_lr1= lr1.predict(x_ts)
pred_tree1= tree1.predict(x_ts)
pred_rf1= rf1.predict(x_ts)
pred_ada1=ada1.predict(x_ts)
pred_gb1=gb1.predict(x_ts)

mode1=[]
for i in np.arange(y_ts.shape[0]):
    votes=np.array([pred_knn1[i], pred_svm1[i], pred_sgd_m1[i],pred_lr1[i],
                    pred_tree1[i], pred_rf1[i], pred_ada1[i], pred_gb1[i]])
    bincount=np.bincount(votes)
    mode1=np.append(mode1,np.argmax(bincount)) #mode is the most common vote, the majority vote


majority_vote_results=majority_vote_results.append(pd.DataFrame([['1st resampled',
            accuracy_score(y_ts, mode1),precision_score(y_ts, mode1,average='macro'),
            recall_score(y_ts, mode1,average='macro'),f1_score(y_ts, mode1,average='macro')]],
            columns=list(majority_vote_results.columns)))
majority_vote_results

### Training Models (on second resampled training set, with synthetic data)

In the test set there are no synthetic data. Only the training set contains 20% of synthetic data.

In [ ]:
#Naive Bayes
gnb2=GaussianNB()
gnb2.fit(x_tr2,y_tr2)
y_pred=gnb2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Gaussian NB', 'default', accuracy,precision,recall,f1]],columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]


In [ ]:
#KNN
knn2=KNeighborsClassifier()
knn2.fit(x_tr2,y_tr2)
y_pred=knn2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['KNN', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#SVM
svm2=SVC()
svm2.fit(x_tr2,y_tr2)
y_pred=svm2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['SVM', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#SGD
sgd2=SGDClassifier()
sgd2.fit(x_tr2,y_tr2)
y_pred=sgd2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['SGD', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#SGD with different parameters
sgd_m2=SGDClassifier(alpha=0.01, loss='modified_huber', penalty='l1')
sgd_m2.fit(x_tr2,y_tr2)
y_pred=sgd_m2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['SGD 2', 'alpha=0.01, loss=modified_huber, penalty=l1', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#Logistic Regression
lr2=LogisticRegression()
lr2.fit(x_tr2,y_tr2)
y_pred=lr2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Logistic Regression', 'default', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#Decision Tree
tree2=DecisionTreeClassifier(max_depth=5, random_state=42)
tree2.fit(x_tr2,y_tr2)
y_pred=tree2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Decision Tree', 'max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#Random Forest
rf2=RandomForestClassifier(n_estimators=1000, max_depth=7, random_state=42)
rf2.fit(x_tr2,y_tr2)
y_pred=rf2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Random Forest', 'n_estimators=1000, max_depth=7', accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#Ada Boost
tree_ada2=DecisionTreeClassifier(criterion='entropy',random_state=1, max_depth=5)
ada2=AdaBoostClassifier(base_estimator=tree_ada2,n_estimators=1000,learning_rate=0.1, random_state=5)
ada2.fit(x_tr2,y_tr2)
y_pred=ada2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Ada Boost', 'criterion=entropy, max_depth=5', accuracy,precision,recall,f1]],
                                    columns=list(results1.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
#Gradient Boosting Tree
gb2=GradientBoostingClassifier(random_state=5, n_estimators=1000, max_depth=4)
gb2.fit(x_tr2,y_tr2)
y_pred=gb2.predict(x_ts)
accuracy=accuracy_score(y_ts,y_pred)
precision=precision_score(y_ts,y_pred, average='macro')
recall=recall_score(y_ts,y_pred, average='macro')
f1=f1_score(y_ts,y_pred, average='macro')
results2=results2.append(pd.DataFrame([['Gradient Boosted Classifier', 'n_estimators=1000, max_depth=4',
                                      accuracy,precision,recall,f1]],
                                    columns=list(results2.columns)))
sns.heatmap(confusion_matrix(y_ts,y_pred),xticklabels= enc.classes_,
            yticklabels=enc.classes_, annot=True,fmt='d');
plt.ylabel('true')
plt.xlabel('predicted');
results2.iloc[-1:]

In [ ]:
results2=results2.reset_index().drop(columns='index')
results2

Clearly the third dataset with the synthetic data didn't perform as well. Let's try a majority vote.

In [ ]:
# majority vote implementation
pred_knn2 = knn2.predict(x_ts) 
pred_svm2 = svm2.predict(x_ts)
pred_sgd_m2= sgd_m2.predict(x_ts)
pred_lr2= lr2.predict(x_ts)
pred_tree2= tree2.predict(x_ts)
pred_rf2= rf2.predict(x_ts)
pred_ada2=ada2.predict(x_ts)
pred_gb2=gb2.predict(x_ts)

mode2=[]
for i in np.arange(y_ts.shape[0]):
    votes=np.array([pred_knn2[i], pred_svm2[i], pred_sgd_m2[i],pred_lr2[i],
                    pred_tree2[i], pred_rf2[i], pred_ada2[i], pred_gb2[i]])
    bincount=np.bincount(votes)
    mode2=np.append(mode2,np.argmax(bincount)) #mode is the most common vote, the majority vote


majority_vote_results=majority_vote_results.append(pd.DataFrame([['2nd resampled plus synthetic',
            accuracy_score(y_ts, mode2),precision_score(y_ts, mode2,average='macro'),
            recall_score(y_ts, mode2,average='macro'),f1_score(y_ts, mode2,average='macro')]],
            columns=list(majority_vote_results.columns)))
majority_vote_results

In [ ]:
display(results)
display(results1)
display(results2)

It seems that the best performance on the test set comes from Gradient Boosting Classifier trained on the original, imbalanced training data. 

Whereas other models were caused to fail by the imbalanced data, Gradient Boosting Tree and Random Forest didn't meet any problems and in fact, they performed better with the original than with the upsampled training sets.